In [1]:
import subprocess
import os
import sys
import pyhmmer
from pyhmmer import hmmscan as hmmscan
import pandas as pd
import multiprocessing.pool
import math
import re
import time
import numpy as np

In [3]:
quer1 = "mycontig_1234_1"
pos = quer1.rfind("_")
contig = quer1[:pos]

print(quer1, pos, contig)

mycontig_1234_1 13 mycontig_1234


In [23]:
### process hallmark taxonomy for ORF calling

mmseqs2_tax_table = "/Users/michaeltisza/mike_tisza/sandbox/ct2_test_728e/ct2_tmp/hallmark_tax/orig_hallmarks_align.tsv"

tax_df = pd.read_csv(mmseqs2_tax_table, header = None, sep = "\t",
                     names = ["query","target","pident","alnlen","evalue","theader","taxlineage"])\
                     .sort_values('evalue').drop_duplicates('query').query("evalue <= 1e-3")

tax_df['ORFcaller'] = np.where(tax_df['taxlineage']
                               .str.contains(
                                   "Crassvirales|Malgrandaviricetes|Tubulavirales|Leviviricetes|Duplopiviricetes|Kalamavirales|Vinavirales|Autolykiviridae",
                                   case = False), 'phanotate', 'prodigal')

tax_df["pos"] = tax_df["query"].str.rfind("_")

tax_df["contig"] = tax_df.apply(lambda x: x["query"][0:x["pos"]], axis = 1)

ORFcaller_majority = tax_df.groupby("contig")['ORFcaller'].agg(pd.Series.mode).to_frame()

prodigal_seqs = ORFcaller_majority.query("ORFcaller == 'prodigal'")



if not prodigal_seqs.empty:
    prodigal_file = "/Users/michaeltisza/mike_tisza/sandbox/prodigal_seqs1.txt"
    if os.path.isfile(prodigal_file):
        os.remove(prodigal_file)

    for i in prodigal_seqs.index:

        print(i, file = open(prodigal_file, "a"))


phanotate_seqs = ORFcaller_majority.query("ORFcaller == 'phanotate'")

if not phanotate_seqs.empty:
    phanotate_file = "/Users/michaeltisza/mike_tisza/sandbox/phanotate_seqs1.txt"
    if os.path.isfile(phanotate_file):
        os.remove(phanotate_file)

    for i in phanotate_seqs.index:
        print(i, file = open(phanotate_file, "a"))



In [9]:
print(tax_df['taxlineage'])

52                                            d_Viruses
33    d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...
14    d_Viruses;-_Duplodnaviria;k_Heunggongvirae;p_U...
65    d_Viruses;-_Monodnaviria;k_Shotokuvirae;p_Cres...
73    d_Viruses;-_Varidnaviria;k_Bamfordvirae;p_Nucl...
0     d_Viruses;-_Monodnaviria;k_Sangervirae;p_Phixv...
Name: taxlineage, dtype: object
